In [1]:
%run ../cube/Cube.ipynb

Support algs imported!
Cube initiated


In [ ]:
# we want to gen algs using U and R' F* R, where F* is any F move
# to do this, we generate a bunch of algs of such form, then generate the cases and save them in a table with the inverse alg
# from any given case, we can loop through the algs and see if we can find a match in the table, if so we have a solution

In [2]:
# generate algs

@njit
def _inc_special(ids):
    for i in range(len(ids)):
        ids[i]+=1
        ids[i]%=6 # change based on number of moves
        if ids[i]:
            break
    return ids

@njit
def _is_valid_special(length,ids):
    for i in range(length-1):
        if ids[i]//3==ids[i+1]//3:
            return False
    return True      

@njit
def _increment_special(length,ids):
    ids = _inc_special(ids)
    while not _is_valid_special(length,ids):
        ids = _inc_special(ids)
    return ids

length = 4
ids = np.array([0,3,0,3])
_increment_special(length,ids)


array([1, 3, 0, 3])

In [5]:
class alg_index_special:
    def __init__(self, length):
        '''
        length: int
        '''
        self.length = length
        self.moves = ["U","U'","U2","R' F R","R' F' R","R' F2 R"] # change to whatever you want to gen # ,"F","F'","F2"
        self.ids = np.array(([0,3]*int(length/2+1))[:length]) # change based on type of algs you gen
    
    def is_valid(self):
        return _is_valid_special(self.length,self.ids)
    
    def inc(self):
        ids = _inc_special(ids)
    
    def increment(self):
        self.ids = _increment_special(self.length,self.ids)
    
    def get_alg(self):
        scr = self.moves[self.ids[0]]
        for i in range(1,len(self.ids)):
            scr += f" {self.moves[self.ids[i]]}"
        return scr
    
ai = alg_index_special(4)
ai.get_alg()

"U R' F R U R' F R"

In [145]:
def gen_algs(n, print_time = True):

    algs = []

    t = time.time()

    for i in range(n):
        ai = alg_index_special(i+1)
        start_alg = ai.get_alg()

        algs.append(start_alg)
        ai.increment()

        while start_alg != ai.get_alg():
            algs.append(ai.get_alg())
            ai.increment()

        if print_time:
            print(f"Algs of length {i+1} genned after {round(time.time()-t,3)}s")
    
    return algs

algs = gen_algs(6)
len(algs)

Algs of length 1 genned after 0.0s
Algs of length 2 genned after 0.0s
Algs of length 3 genned after 0.001s
Algs of length 4 genned after 0.001s
Algs of length 5 genned after 0.003s
Algs of length 6 genned after 0.01s


2184

In [144]:
def gen_table(algs):
    '''
    Gen dictionary with keys being cases reached by inverting each alg, with dict[case] = alg
    '''
    table = {}

    for alg in algs:
        inv = inverse_alg(alg)
        # skip algs ending with U* as the first move is equivalent to a AUF
        if alg[0]=="U":
            continue

        cube = Cube() # only F2L
        cube.apply_moves(alg)
        case = id_from_state(cube.state)

        if case not in table.keys():
            table[case] = [inv]
        else:
            table[case] += [inv]

    return table

table = gen_table(algs)
len(table)

78504

In [78]:
def gen_random_alg_from_move_list(length, moves = ["U","U'","U2","R' F R","R' F' R","R' F2 R"]):
    return cancel_alg(" ".join(np.random.choice(moves,length)))

In [253]:
cube = Cube()

# cube.apply_moves("R2 B2 R2 D U L2 R2 U R2 F2 L F U L' U' F' D' U F'")
# cube.apply_moves("F2 D2 F D2 L U' L U2 B' U' B L F' L'")
# cube.apply_moves("U2 L' U L")
sol_lengths = []
cube.apply_moves("R' U L' U2 R U' L R' U L' U2 R U' L")
cube.apply_moves("F R U R' U' F' L' U2 L")

# cube.apply_moves("M' U2 M")

# new_alg = gen_random_alg_from_move_list(500)
# print("scramble:",new_alg)
# cube.apply_moves(new_alg)
# cube.plot()

sol = ""
for alg in algs:
    cube.apply_moves(alg)
    case = id_from_state(cube.state)
    if case in table.keys():
        sol = cancel_alg(alg + " " + table[case][0])
        sol_len = len(sol.split(" "))
        print(f"{sol} ({sol_len})")
        break
    cube.apply_moves(inverse_alg(alg))

U2 R' F R U2 R' F' R U' R' F' R U' R' F R U' R' F2 R U2 R' F2 R U2 R' F' R (28)


In [ ]:
# alg lengths (over 10 000 scrambles): 27.5772